In [1]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-09-25 16:26:43.926570


In [1]:
!git clone https://github.com/abinesh1/EVA5.git

fatal: destination path 'EVA5' already exists and is not an empty directory.


In [2]:
%cd EVA5/S9

/content/EVA5/S9


In [3]:
import torch
#import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms

!pip install torchsummary
from torchsummary import summary

from ezio.data import dataset 
from ezio.runner import engine
from ezio.utils import loss

In [4]:
## Create train and test transforms
train_transforms = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transforms = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
train_set, valid_set = dataset.cifar10_dataset(train_transforms, test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader, valid_loader = dataset.create_loaders(train_set, valid_set, train_batch_size=128, valid_batch_size=128, num_workers=4)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class QuizDNN1(nn.Module):
	def __init__(self):
		super(QuizDNN1, self).__init__()
		self.conv = nn.Sequential(nn.Conv2d(3, 64, 3, padding = 1, bias = False),
			nn.ReLU(),
			nn.BatchNorm2d(64))
		self.conv2 = nn.Sequential(nn.Conv2d(64, 64, 3, padding = 1, bias = False),
			nn.ReLU(),
			nn.BatchNorm2d(64),
			nn.Dropout(0.1))
		self.pool1 = nn.MaxPool2d(2,2)
		self.gap = nn.Sequential(nn.AvgPool2d(1))
			#nn.BatchNorm2d(64))
		self.fully_conn = nn.Sequential(nn.Conv2d(64, 10, 1, padding = 0, bias = False))

	def forward(self, x):
		x1 = self.conv(x)
		x2 = self.conv2(x1)
		x3 = self.conv2(x1+x2)
		x4 = self.pool1(x1+x2+x3)
		x5 = self.conv2(x4)
		x6 = self.conv2(x4+x5)
		x7 = self.conv2(x4+x5+x6)
		x8 = self.pool1(x5+x6+x7)
		x9 = self.conv2(x8)
		x10 = self.conv2(x8+x9)
		x11 = self.conv2(x8+x9+10)
		x12 = self.gap(x11)
		x13 = self.fully_conn(x12)
		x = x13.view(-1, 10)

		return x

In [9]:
from ezio.model.QuizDNN import QuizDNN
model = QuizDNN().to(device)

In [10]:
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
            Conv2d-4           [-1, 64, 32, 32]          36,864
              ReLU-5           [-1, 64, 32, 32]               0
       BatchNorm2d-6           [-1, 64, 32, 32]             128
           Dropout-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
              ReLU-9           [-1, 64, 32, 32]               0
      BatchNorm2d-10           [-1, 64, 32, 32]             128
          Dropout-11           [-1, 64, 32, 32]               0
        MaxPool2d-12           [-1, 64, 16, 16]               0
           Conv2d-13           [-1, 64, 16, 16]          36,864
             ReLU-14           [-1, 64,

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=0, verbose=True, factor=0.5)
loss_function = loss.cross_entropy_loss()

In [12]:
train_acc, train_losses, test_acc, test_losses = engine.fit(model, train_loader, valid_loader, loss_function, device, optimizer, 40)

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 1


Loss=1.2034573554992676 Batch_id=390 Accuracy=43.00: 100%|██████████| 391/391 [00:15<00:00, 25.74it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.1907, Accuracy: 1656/10000 (16.56%)

EPOCH: 2


Loss=0.9901202917098999 Batch_id=390 Accuracy=57.00: 100%|██████████| 391/391 [00:15<00:00, 25.84it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

...


Test set: Average loss: 0.1286, Accuracy: 1001/10000 (10.01%)

EPOCH: 39


Loss=0.6941468715667725 Batch_id=390 Accuracy=75.00: 100%|██████████| 391/391 [00:14<00:00, 27.31it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.2506, Accuracy: 1000/10000 (10.00%)

EPOCH: 40


Loss=0.6453280448913574 Batch_id=390 Accuracy=75.00: 100%|██████████| 391/391 [00:14<00:00, 27.06it/s]



Test set: Average loss: 0.1513, Accuracy: 1287/10000 (12.87%)

